In [192]:
#pip install tensorflow

# Importing libraries and dataset

In [158]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [160]:
tf.__version__

'2.15.0'

In [161]:

dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values 
y = dataset.iloc[:, -1].values 

# Part 1 - Data Preprocessing

## Encoding categorical data

### Label enconding the "Gender" column 

In [186]:
import sklearn 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])


In [163]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


## one hot encoding the "Geography" column

In [164]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder() ,[1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))


print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [165]:
x.shape

(10000, 12)

In [166]:
y.shape 

(10000,)

## Splitting the dataset into the training and test sets 

In [167]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)


In [168]:
x_train.shape

(8000, 12)

In [169]:
y_train.shape 

(8000,)

## Feature Scaling 

In [170]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train= sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)



# Part2 - Building the ANN

## Initializing the ANN

In [171]:
ann = tf.keras.models.Sequential()


## Adding the input layer and the first hidden layer

In [172]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))


## Adding the second input layer

In [173]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))


## Adding the output layer 

In [174]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the ANN

## Compiling the ANN

In [175]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Training the ANN on the training set 

In [176]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5765 - accuracy: 0.7519
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4757 - accuracy: 0.7958
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4477 - accuracy: 0.8015
Epoch 4/100
 84/250 [=========>....................] - ETA: 0s - loss: 0.4358 - accuracy: 0.8118

250/250 [==============================] - 0s 1ms/step - loss: 0.4327 - accuracy: 0.8136
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4244 - accuracy: 0.8207
Epoch 6/100
250/250 [==============================] - 0s 957us/step - loss: 0.4189 - accuracy: 0.8230
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4144 - accuracy: 0.8251
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4101 - accuracy: 0.8267
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4059 - accuracy: 0.8259
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4017 - accuracy: 0.8280
Epoch 11/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3980 - accuracy: 0.8290
Epoch 12/100
250/250 [==============================] - 0s 961us/step - loss: 0.3946 - accuracy: 0.8285
Epoch 13/100
250/250 [==============================] - 0s 953us/step - loss: 0.3914 - accuracy:

# Part 4 - Making the predictions and evaluating the model 

## Predicting the result of a single observation 

In [177]:
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

1/1 [==============================] - 0s 178ms/step


array([[0.03273582]], dtype=float32)

In [179]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))>0.5)

1/1 [==============================] - 0s 24ms/step
[[False]]


## Predicting the test set results 

In [184]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)), 1))

 1/63 [..............................] - ETA: 1s

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the confusion matrix 

In [191]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1527   68]
 [ 212  193]]


0.86